# Electricistan

Problem to be solved: Datetime (yyyy.mm.dd hh:mm:00) is given and power-consumption for this given point in time needs to be predicted/forecasted for the next two weeks. 
"Given past data, predict power-consumption for the two weeks ahead for a selected point in time." 
Root Mean Squared Error (RMSE) is being used - between predicted value and observed power consumption. 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose

# use inline backend for figures - default.
%matplotlib inline   

In [2]:
# read data and have a first look at the beginning and at the end. 
orig_data = pd.read_csv("../data/electricistan/train.csv")

In [3]:
print(orig_data.head(100))

               datetime  power
0   2012.01.01 00:15:00   3767
1   2012.01.01 00:30:00   3743
2   2012.01.01 00:45:00   3691
3   2012.01.01 01:00:00   3645
4   2012.01.01 01:15:00   3628
..                  ...    ...
95  2012.01.02 00:00:00   3472
96  2012.01.02 00:15:00   3395
97  2012.01.02 00:30:00   3323
98  2012.01.02 00:45:00   3265
99  2012.01.02 01:00:00   3224

[100 rows x 2 columns]


In [4]:
# explore data quickly. No missing data can be detected at a first look. But datetime is of type object instead of datetime.
# more than 370'000 records of data available.
orig_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375355 entries, 0 to 375354
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   datetime  375355 non-null  object
 1   power     375355 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 5.7+ MB


In [5]:
# convert datetime from object to datetime format. 
orig_data['datetime'] = pd.to_datetime(orig_data['datetime'])

## Explore Data

In [ ]:
# No missing data for datetime and power - null/nan values. 
orig_data.isna().sum()

In [ ]:
# Get a first data insight regarding descriptive stats
orig_data.describe()

In [ ]:
# plot data a first time. Result is too dense and no valuable info can be seen...
sns.lineplot(orig_data, x="datetime", y="power")

In [ ]:
# resize figure, set style and add zooming option to get more insights.
plt.figure(figsize=(16,7))
sns.set_style('darkgrid')
sns.lineplot(data=orig_data, x='datetime', y='power')
plt.show()

px.line(orig_data, x='datetime', y='power')

In [ ]:
# check distribution of 'power' and if there are any outliers
fig, ax = plt.subplots(figsize=(4,3))
ax.boxplot(orig_data.power, vert=False)
ax.set_xlabel("power")
plt.show()

In [ ]:
# Have a look at the histogram (10 bins) data is --not-- distributed normally. 
fig, ax = plt.subplots(figsize=(4,3))
sns.histplot(orig_data.power, bins=10, kde=True)
ax.set_xlabel("number of values")
ax.set_ylabel("power")
plt.title("Histogram with 10 bins")
plt.show()

In [ ]:
# check ranges selected for the 10 bins and respective amount of 'power' values for each bin. 
np.histogram(orig_data.power, bins=10)

In [6]:
# enhance dataset by date on a day basis - attach timestamp to a single day
temp_data = orig_data.copy()
temp_data['date'] = pd.DatetimeIndex(temp_data['datetime']).date
# count number of days in the dataset.
temp_data['date'].nunique()   

3910

In [ ]:
# Timeseries decompose - level, trend, seasonality, noise
# use a year for period = 4 (values per hour) * 24 (hours) * 365 (days) = 35040
data_decomposed = seasonal_decompose(x=temp_data.power, model="additive", period=35040)
trend_estimate = data_decomposed.trend
seasonal_estimate = data_decomposed.seasonal
residual_estimate = data_decomposed.resid

In [ ]:
fig, axes = plt.subplots(4, 1, sharex=True, sharey=False)
fig.set_figheight(10)
fig.set_figwidth(20)
#Original data
axes[0].plot(temp_data.power, label='Original')
axes[0].legend(loc='upper left')
#Trend
axes[1].plot(trend_estimate, label='Trend')
axes[1].legend(loc='upper left')
#Sesonality
axes[2].plot(seasonal_estimate, label='Seasonality')
axes[2].legend(loc='upper left')
#Residuals
axes[3].plot(residual_estimate, label='Residuals')
axes[3].legend(loc='upper left')

## Prepare Data

In [7]:
df_temp = orig_data.copy()

In [8]:
df_temp = df_temp.set_index('datetime')
df_temp.index = pd.to_datetime(df_temp.index)
df_temp.head()

,power
datetime,
2012-01-01 00:15:00,3767
2012-01-01 00:30:00,3743
2012-01-01 00:45:00,3691
2012-01-01 01:00:00,3645
2012-01-01 01:15:00,3628


In [9]:
# Feature creation
df_temp['hour'] = df_temp.index.hour
df_temp['dayofweek'] = df_temp.index.dayofweek
df_temp['quarter'] = df_temp.index.quarter
df_temp['month'] = df_temp.index.month
df_temp['year'] = df_temp.index.year
df_temp['dayofyear'] = df_temp.index.dayofyear
df_temp['dayofmonth'] = df_temp.index.day
df_temp['weekofyear'] = df_temp.index.isocalendar().week

In [ ]:
df_temp.head()
# dayofweek 6 = Sun ... as 01.01.2012 was a Sun...

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
sns.boxplot(data=df_temp, x='month', y='power')
ax.set_title('power by month')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
sns.boxplot(data=df_temp, x='hour', y='power', palette='Blues')
ax.set_title('power by hour')
plt.show()

In [11]:
# time-step feature added
df_temp['Time'] = np.arange(len(df_temp.index))
df_temp.head()

,power,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,Time
datetime,,,,,,,,,,
2012-01-01 00:15:00,3767,0,6,1,1,2012,1,1,52,0
2012-01-01 00:30:00,3743,0,6,1,1,2012,1,1,52,1
2012-01-01 00:45:00,3691,0,6,1,1,2012,1,1,52,2
2012-01-01 01:00:00,3645,1,6,1,1,2012,1,1,52,3
2012-01-01 01:15:00,3628,1,6,1,1,2012,1,1,52,4


In [13]:
# lag feature added
df_temp['Lag_1'] = df_temp['power'].shift(1)
df_temp.head()

,power,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,Time,Lag_1
datetime,,,,,,,,,,,
2012-01-01 00:15:00,3767,0,6,1,1,2012,1,1,52,0,NaN
2012-01-01 00:30:00,3743,0,6,1,1,2012,1,1,52,1,3767.0
2012-01-01 00:45:00,3691,0,6,1,1,2012,1,1,52,2,3743.0
2012-01-01 01:00:00,3645,1,6,1,1,2012,1,1,52,3,3691.0
2012-01-01 01:15:00,3628,1,6,1,1,2012,1,1,52,4,3645.0


In [16]:
# drop missing values
df_temp.dropna(inplace=True)
df_temp.head()

,power,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,Time,Lag_1
datetime,,,,,,,,,,,
2012-01-01 00:30:00,3743,0,6,1,1,2012,1,1,52,1,3767.0
2012-01-01 00:45:00,3691,0,6,1,1,2012,1,1,52,2,3743.0
2012-01-01 01:00:00,3645,1,6,1,1,2012,1,1,52,3,3691.0
2012-01-01 01:15:00,3628,1,6,1,1,2012,1,1,52,4,3645.0
2012-01-01 01:30:00,3558,1,6,1,1,2012,1,1,52,5,3628.0


## Additional Data
Additional data that might be interesting and influencing power consumption and therefore needs to be added to the dataframe. 

In [ ]:
# Vacation - depending on region
# Weather?
# Corona?

# Write Data

In [ ]:
# write data to csv and train from there
df_temp.to_csv("../data/electricistan/train_prepped.csv")